<a href="https://colab.research.google.com/github/ClarinetInALeatherJacket/OgataLabROP2024/blob/main/MeanStdAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import codecs
import time
#!pip install numpy
import numpy as np
#!pip install pandas
import pandas as pd
#!pip install matplotlib
import matplotlib.pyplot as plt
from google.colab import files
%load_ext google.colab.data_table
#!pip install ipdb
#import ipdb
#%pdb off

In [ ]:
# Store the names of the columns in an array for easy access
columns = ["freq / Hz", "Z' / Ohm", "Z'' / Ohm", "None"]

# Custom class to store the data. Stores a name, code, and any data, and
# automatically generates the mean and standard deviations
class Experiment:
    code: str
    name: str
    data = []
    mean = pd.DataFrame
    std = pd.DataFrame

    def __init__(self, code: str, name: str):
        self.code = code
        self.name = name
        self.data = []
        self.mean = pd.DataFrame
        self.std = pd.DataFrame

    def add_data(self, data):
        self.data.append(data)
        # Any time new data is added, update the mean and deviation. If there is
        # only one trial and the deviation comes up as NANs, fill w/ 0
        self.mean = pd.concat(self.data).groupby("freq / Hz").mean()
        self.std = pd.concat(self.data).groupby("freq / Hz").std()
        self.std = self.std.fillna(value=0)


In [ ]:
def clear_temp_dirs():
  """
  Empties and deletes the ./Input and ./Converted directories created during
  parse_input().
  """
  if os.path.exists("./Input"):
    for file in os.listdir("./Input"):
      os.remove(f"./Input/{file}")
    os.rmdir("./Input")

  if os.path.exists("./Converted"):
    for file in os.listdir("./Converted"):
      os.remove(f"./Converted/{file}")
    os.rmdir("./Converted")

  if os.path.exists("./Output"):
    for file in os.listdir("./Output"):
      os.remove(f"./Output/{file}")
    os.rmdir("./Output")


In [ ]:
def parse_input()->list[Experiment]:
  """
  Takes in a single CSV file with multiple experiments and returns a list of
  experiment data structures, containing the numeric data and ID codes
  """
  # Get the file name for the inputted file
  input_file = list(files.upload().keys())[0]

  # If the Input and Converted directories don't exist, create them
  if not os.path.exists("./Input"):
    os.mkdir("./Input")
  if not os.path.exists("./Converted"):
    os.mkdir("./Converted")
  if not os.path.exists("./Output"):
    os.mkdir("./Output")

  # Move the input file to the input directory
  shutil.move(f"./{input_file}",f"./Input/{input_file}")

  # Convert input file from UTF16 LE BOM encoding to UTF8
  with codecs.open(f"./Input/{input_file}", "r", encoding="utf-16") as f:
    with codecs.open(f"./Converted/{input_file}", "w",encoding="utf-8") as f2:
      shutil.copyfileobj(f, f2)

  output = []

  # Read the input file and turn it into an array of lines
  with open(f"./Converted/{input_file}") as file:
      lines = file.read().splitlines()

  # Start a counter to iterate through all the input file lines and a set to
  # track which codes have been encountered before
  i = 0
  seen_codes = set()


  while i < len(lines) - 1:

    # If a line starting with "Measurement", in other words the beginning of
    # a data set header, is found, begin processing the data set
    if lines[i].startswith("Measurement"):

      # Extract the name and code from the "Measurement" line
      split = lines[i].split(",")[1].split(".")
      code = split[0]
      name = split[1]

      # If the code has not been encountered before, create an experiment for
      # that code
      if code not in seen_codes:
        x = Experiment(code, name)
        output.append(x)
        seen_codes.add(code)

      # Locate the index of the line that immediately precedes the numeric data
      while not lines[i].startswith("freq"):
        i += 1

      start = i

      # Locate the index of the empty line that immediately succeeds the numeric
      # data
      while i < len(lines) - 1 and not lines[i + 1] == "":
        i += 1

      end = i

      # Locate the experiment that coincides with this numeric data's code, then
      # add the data to that experiment
      for j in output:
        if j.code == code:
          temp = pd.read_csv(f"./Converted/{input_file}",
                              names=columns,
                              skiprows=start + 1,
                              nrows = end - start)
          temp = temp.dropna(axis=1) # Drop an extra column that is formed
                                     # due to CSV file format
          j.add_data(temp)

      i += 1

    else:
      i += 1

  return output

In [ ]:
if __name__ == "__main__":
    clear_temp_dirs()
    experiments = parse_input()
    experiments.sort(key=lambda y: int(y.code))

    temp = []

    for x in experiments:
        temp.append(pd.concat([x.mean, x.std], keys=["mean", "std dev"], axis=1))

    output = pd.concat(temp, keys=[x.name for x in experiments], axis=1).sort_index(ascending=False)

    name = input("What would you like to name the output file? ")

    output.to_excel(f"{name}.xlsx")

    files.download(f"{name}.xlsx")

    clear_temp_dirs()

Saving Observer_Contract_Signed.pdf to Observer_Contract_Signed.pdf
Saving Observer_Contract.pdf to Observer_Contract.pdf
Saving IMS100_EN_PDFReading.pdf to IMS100_EN_PDFReading.pdf
Saving IMS100_Cert.pdf to IMS100_Cert.pdf
Saving Fillable PCR.pdf to Fillable PCR.pdf
Saving Attestation Form 2024.pdf to Attestation Form 2024.pdf
Saving 2024 Frosh_Orientation Availability Form.xlsx to 2024 Frosh_Orientation Availability Form.xlsx


UnicodeDecodeError: 'utf-16-le' codec can't decode bytes in position 486-487: illegal UTF-16 surrogate